In [1]:
from pathlib import Path

from rich import print

import opera_utils.disp.search

%matplotlib inline
%load_ext rich

In [2]:
frame_id = 11116
url_type = "https"
num_workers = 5
work_dir = Path("/home/staniewi/dev/disp-production-data/descending/F11116")

In [9]:
granules = opera_utils.disp.search.get_products(frame_id=frame_id, url_type=url_type)
granules = sorted(granules, key=lambda g:g.product.secondary_datetime)
print(f"Found {len(granules) = } for {frame_id = }")
print(granules[:2])

Found len(granules) = 299 for frame_id = 11116

[
    Granule(
        product=DispProduct(
            filename='https://datapool.asf.alaska.edu/DISP/OPERA-S1/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_
20160729T140756Z_v1.0_20250408T163512Z.nc',
            sensor='S1',
            acquisition_mode='IW',
            frame_id=11116,
            polarization='VV',
            reference_datetime=datetime.datetime(2016, 7, 5, 14, 7, 55, tzinfo=datetime.timezone.utc),
            secondary_datetime=datetime.datetime(2016, 7, 29, 14, 7, 56, tzinfo=datetime.timezone.utc),
            version='1.0',
            generation_datetime=datetime.datetime(2025, 4, 8, 16, 35, 12, tzinfo=datetime.timezone.utc)
        ),
        url='https://datapool.asf.alaska.edu/DISP/OPERA-S1/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160729T
140756Z_v1.0_20250408T163512Z.nc',
        orbit_pass=<OrbitPass.DESCENDING: 'DESCENDING'>
    ),
    Granule(
        product=DispProduct(
            filename='https://datapool.asf.alaska.edu/DISP/OPERA-S1/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_
20160810T140757Z_v1.0_20250408T163512Z.nc',
            sensor='S1',
            acquisition_mode='IW',
            frame_id=11116,
            polarization='VV',
            reference_datetime=datetime.datetime(2016, 7, 5, 14, 7, 55, tzinfo=datetime.timezone.utc),
            secondary_datetime=datetime.datetime(2016, 8, 10, 14, 7, 57, tzinfo=datetime.timezone.utc),
            version='1.0',
            generation_datetime=datetime.datetime(2025, 4, 8, 16, 35, 12, tzinfo=datetime.timezone.utc)
        ),
        url='https://datapool.asf.alaska.edu/DISP/OPERA-S1/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160810T
140757Z_v1.0_20250408T163512Z.nc',
        orbit_pass=<OrbitPass.DESCENDING: 'DESCENDING'>
    )
]

In [3]:
data_dir = work_dir / "ncs"
data_dir.mkdir(exist_ok=True, parents=True)

In [20]:
import pathlib

import requests
from tqdm.contrib.concurrent import process_map


def download_url(url_info: tuple[str, pathlib.Path]) -> pathlib.Path:
    """Download a file from a URL to a specified path.

    Args:
        url_info: tuple containing the URL and output directory path

    Returns:
        Path to the downloaded file
    """
    url, output_dir = url_info
    out_path = output_dir / pathlib.Path(url).name
    if out_path.exists():
        return out_path
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(out_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    return out_path


def download_urls(
    urls: list[str], output_path: pathlib.Path, max_workers: int = 4
) -> list[pathlib.Path]:
    """Download multiple URLs in parallel with a progress bar.

    Args:
        urls: list of URLs to download
        output_path: Directory where files will be saved
        max_workers: Maximum number of parallel processes

    Returns:
        list of paths to the downloaded files
    """
    # Make sure the output directory exists
    output_path.mkdir(exist_ok=True, parents=True)

    # Prepare input for the download function (url and output path)
    url_infos = [(url, output_path) for url in urls]

    # Download files in parallel with progress bar
    results = process_map(
        download_url,
        url_infos,
        max_workers=max_workers,
        desc="Downloading files",
        unit="file",
    )

    return results


In [12]:
%%time

download_urls([g.url for g in granules], output_path=data_dir, max_workers=num_workers);

CPU times: user 470 ms, sys: 233 ms, total: 703 ms
Wall time: 11min 51s



[
    PosixPath('/home/staniewi/dev/disp-production-data/descending/F11116/ncs/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160729T140756Z_v1.0_20250408T163512Z.nc'),
    PosixPath('/home/staniewi/dev/disp-production-data/descending/F11116/ncs/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160810T140757Z_v1.0_20250408T163512Z.nc'),
    PosixPath('/home/staniewi/dev/disp-production-data/descending/F11116/ncs/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160822T140758Z_v1.0_20250408T163512Z.nc'),
    PosixPath('/home/staniewi/dev/disp-production-data/descending/F11116/ncs/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160903T140758Z_v1.0_20250408T163512Z.nc'),
    PosixPath('/home/staniewi/dev/disp-production-data/descending/F11116/ncs/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160915T140759Z_v1.0_20250408T163512Z.nc'),
    PosixPath('/home/staniewi/dev/disp-production-data/descending/F11116/ncs/OPERA_L3_DISP-S1_IW_F11116_VV_20160705T140755Z_20160927T140759Z_v1.0_202504

In [4]:
# TODO: move script to something importable 

# %run -n /home/staniewi/repos/opera-utils/scripts/rebase_reference.py
import sys

sys.path.append("/home/staniewi/repos/opera-utils/scripts/")
import importlib

import rebase_reference

importlib.reload(rebase_reference)

<module 'rebase_reference' from '/home/staniewi/repos/opera-utils/scripts/rebase_reference.py'>

In [5]:
nc_files = sorted(data_dir.glob("*.nc"))
output_dir = work_dir / "aligned"
output_dir.mkdir(exist_ok=True)

In [22]:
%%time

rebase_reference.main(nc_files=nc_files, output_dir=output_dir, apply_corrections=True, reference_point=None, num_workers=num_workers)

CPU times: user 990 ms, sys: 725 ms, total: 1.72 s
Wall time: 26min 54s


In [6]:
from dolphin import timeseries, utils

In [7]:
timeseries.create_velocity

<function create_velocity at 0x7f2536aa8ae0>

In [10]:
%%time
utils.disable_gpu()

disp_files = sorted(output_dir.glob("displacement*tif"))
velocity_file = output_dir / "velocity.tif"
timeseries.create_velocity(unw_file_list=disp_files, output_file=velocity_file, reference=None, num_threads=5)

  0%|          | 0/1026 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 14min 27s, sys: 4min 50s, total: 19min 18s
Wall time: 2min 40s
